# Vectorisation

One of the promises of JAX is to make vectorisation great again via the use of syntactic sugar decorators that describe what inputs are batched onto what outputs. The goal of this notebook is to show how this can be done in practice as well as how this is translated in terms of low-level code. 

## JAX imports

## Beginner
### Prerequisites
NumPy - (some exposure to Numba is helpful)

### Imports

In [2]:
import inspect 

from jax import vmap, make_jaxpr
import jax.numpy as jnp

import numpy as np

### Example

To compare the vectorisation implementation of JAX to the NumPy one, let's take the following example:

In [3]:
def indexing_function(x, y):
    # Here x is a vector of floats, and y is a vector of ints
    return x[y]

We will use the following array for our tests:

In [5]:
N = 10

In [6]:
indexing_function(np.random.randn(N), np.random.randint(N))

-2.411996272102251

How does it react to batched inputs?

In [7]:
B = 3

In [8]:
indexing_function(np.random.randn(B, N), np.random.randint(N, size=B))

IndexError: index 8 is out of bounds for axis 0 with size 3

In [9]:
indexing_function(np.random.randn(B, N), np.random.randint(N))

IndexError: index 4 is out of bounds for axis 0 with size 3

OK so we need to modify it.

In [10]:
def complicated_indexing_function(x, y):
    # Here x is a vector of floats, and y is a vector of ints
    return x[..., y]

In [11]:
complicated_indexing_function(np.random.randn(B, N), np.random.randint(N))

array([ 0.11550145, -0.37564915, -0.91943035])

In [13]:
complicated_indexing_function(np.random.randn(B, N), np.random.randint(N, size=B))

array([[-1.36216897, -1.36216897, -1.36216897],
       [ 0.64245125,  0.64245125,  0.64245125],
       [ 0.03967815,  0.03967815,  0.03967815]])

Really not what we want!

### NumPy-style vectorisation

Instead of trying to be smart, let's use NumPy:

In [14]:
np_vectorised_indexing_function = np.vectorize(indexing_function, signature="(n),()->()")

In [15]:
np_vectorised_indexing_function(np.random.randn(B, N), np.random.randint(N))

array([-0.51309691,  1.96632949,  1.3273036 ])

And the JAX vectorisation:

In [16]:
jax_vectorised_indexing_function = jnp.vectorize(indexing_function, signature="(n),()->()")

In [17]:
jax_vectorised_indexing_function(np.random.randn(B, N), np.random.randint(N))

/home/tamire1/.conda/envs/NFSDE/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


DeviceArray([-1.0067911 ,  1.7279607 , -0.25405183], dtype=float32)

So what is the difference?

In [18]:
batch_input = np.random.randn(10000, N)
batch_index = np.random.randint(N, size=10000)

In [19]:
%timeit np_vectorised_indexing_function(batch_input, batch_index)

21.9 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
jax_batch_input = jnp.asarray(batch_input)
jax_batch_index = jnp.asarray(batch_index)

In [21]:
%timeit jax_vectorised_indexing_function(jax_batch_input, jax_batch_index).block_until_ready()

3.29 ms ± 98.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Why is it faster? Because it's multi-threaded in the background!

### Vectorised map

On the other hand one can pick vmap: `jnp.vectorize` is a wrapper around the vmap functionality, this is useful in the case when the batching dimension is not the first one for example.

In [49]:
vmapped_indexing = vmap(indexing_function, in_axes=(1, 0))  
# here we are saying that the input will be batched along 
# the second dimension for the input, and the first for the index, this helps with not having to do shape arithmetics.

In [50]:
vmapped_indexing(np.random.randn(N, 3), np.random.randint(N, size=3))

DeviceArray([-0.21892872, -0.4816025 , -0.32615903], dtype=float32)

### Questions:

#### Q1: 
Reimplement this manually vectorised function using `vmap`, and compare the generated code using `make_jaxpr`

In [24]:
def just_a_function(x, y):
    a = x[..., 0] * y[..., 1]
    b = x[..., 1] * y[..., 0]
    return a + b

In [38]:
x = np.random.randn(4, 3, 2)
y = np.random.randn(4, 3, 2)

In [39]:
just_a_function(x, y)

array([[ 0.7054689 , -0.34309396, -0.2530485 ],
       [-0.40755578, -0.17743709,  0.24736201],
       [ 0.51945306, -0.57256008,  0.0951352 ],
       [ 0.05290742,  0.15531526, -3.1203763 ]])

In [59]:
def just_ab(x, y):
    a = x[0] * y[1]
    b = x[1] * y[0]
    return a + b

vmap_ab = vmap(just_ab, in_axes=(0, 0))
vmap_ab(x, y)

DeviceArray([[-2.4245608 , -0.8565366 ],
             [-0.59892577, -0.12430437],
             [-0.31583315,  1.2837989 ],
             [ 0.33456147,  0.05335914]], dtype=float32)

#### Q2:
Using `jnp.vectorize`, vectorise the following function with respect to the matrix `a`:
```python
def solve(a, b):
    return jnp.solve(a, b)
```

In [72]:
def solve(a, b):
    return jnp.linalg.solve(a, b)

vmap_solve = jnp.vectorize(solve, signature="(m,n),(m)->(n)" )
a = np.random.randn(100, 10, 5)

b = np.random.randn(10)
vmap_solve(a, b)

ValueError: The arguments to solve must have shapes a=[..., m, m] and b=[..., m, k] or b=[..., m]; got a=(10, 5) and b=(10,)

## Intermediate / Advanced
### Prerequisites
- Beginner vectorisation
- Beginner automatic differentiation
- Beginner loops (Advanced)

Now that we know how to vectorise, let's give an example where it's not only just a convenient wrapper but also a useful computational tool: we will see how to vectorise the JVP call we learned about in the automatic differentiation notebook.

### Imports

In [126]:
from functools import partial 

from jax import make_jaxpr, jvp, vmap
import jax.numpy as jnp
from jax.random import normal, PRNGKey

import numpy as np

### Example

Let's take a simple example:

In [115]:
def fun(x):
    return jnp.sin(jnp.sum(x))

Say we want to compute its JVP against a number of random vectors:

In [124]:
def jvp_fun(x, key, d=100):
    n = x.shape[0]
    vectors = normal(key, shape=(n, d))
    return jvp(fun, (x,), (vectors,))

In [125]:
jvp_fun(jnp.array([0., 1.]), PRNGKey(42))

TypeError: Gather op must have one slice size for every input dimension; got: len(slice_sizes)=1, input_shape.rank=2

It doesn't work out of the box it seems... Let's try and obey the syntax of JVP:

In [151]:
def jvp_fun(x, key, d=20):
    n = x.shape[0]
    vectors = normal(key, shape=(n, d))
    return jvp(fun, (jnp.repeat(x.reshape(-1, 1), d, 1),), (vectors,))[1]

In [152]:
jvp_fun(jnp.array([0., 1.]), PRNGKey(42))

DeviceArray([ 0.67665976,  0.29954007, -1.0683215 , -0.22646905,
              0.98523813,  1.1727225 ,  0.40628698,  0.43852165,
              0.5092319 , -0.10380521,  1.348889  , -0.7610773 ,
              0.14572985,  0.07581756,  0.70275015,  1.2393789 ,
             -0.09658202, -1.0514277 ,  0.57944995,  0.21703981],            dtype=float32)

OK it's working so what's the problem here? `fun` is being relinearised at the same point $d$ times for no reason!

You can execute the line below to see this

In [157]:
# make_jaxpr(jvp_fun)(jnp.array([0., 1.]), PRNGKey(42))

We can actually solve this problem by using `vmap`:

In [158]:
def vmap_jvp_fun(x, key, d=20):
    n = x.shape[0]
    vectors = normal(key, shape=(n, d))
    local_fun = lambda vec: jvp(fun, (x,), (vec,))[1]
    return vmap(local_fun, in_axes=(1,))(vectors)

In [159]:
vmap_jvp_fun(jnp.array([0., 1.]), PRNGKey(42))

DeviceArray([ 0.67665976,  0.29954007, -1.0683215 , -0.22646905,
              0.98523813,  1.1727225 ,  0.40628698,  0.43852165,
              0.5092319 , -0.10380521,  1.348889  , -0.7610773 ,
              0.14572985,  0.07581756,  0.70275015,  1.2393789 ,
             -0.09658202, -1.0514277 ,  0.57944995,  0.21703981],            dtype=float32)

Execute the following line to compare with the naive manual approach

In [161]:
# make_jaxpr(vmap_jvp_fun)(jnp.array([0., 1.]), PRNGKey(42))

### Questions:

#### Q1:
Vectorise the following bubble sort algorithm using the method of your choice:
```python
def bubbleSort(arr): 
    n = len(arr) 
    res = np.copy(arr)
    for i in range(n-1): 
        for j in range(0, n-i-1): 
            if res[j] > res[j+1]: 
                res[j], res[j+1] = res[j+1], res[j]
    return res   
```